In [1]:
import pandas as pd

dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')
dataset

,Patient MRN,Anonymized Patient Identifier,Date of Birth,Year of Birth,Date of Death,Age At Date of Death,Cause of Death,Cause of Death Attributable to Treatment,Sex at Birth,Patient Reported Race,...,Staging-N,Staging-M,Imaging for Biopsy,Pathology Molecular Test Name,Pathology Molecular Test Result,Treatment Course ID,Treatment Type,Start Date of Treatment,End Date of Treatment,Treatment Outcome
0,NaN,Anon70998,04/12/1996,NaN,NaN,26.390144,C15,NaN,Male,Other Pacific Islander,...,N0,M1b,Magnetic Resonance Imaging,Oncotype DX Genomic Prostate Score,High risk,TID8305,Surgery,2018-05-06,2024-04-05,Progressive Disease
1,8942976.0,NaN,14/08/1983,1983.0,NaN,35.696099,C15,Definitely Related,Undifferentiated,NaN,...,Nx,M1c,Magnetic Resonance Imaging,DECIPHER,Low risk,TID7003,Surgery,2018-06-15,2023-05-17,Stable Disease
2,NaN,Anon98591,15/09/1952,NaN,NaN,NaN,C15,NaN,Not Disclosed,American Indian or Alaska Native,...,N1,M1c,Computed Tomography,Prolaris,Intermediate risk,TID7466,Surgery,2021-11-11,2023-11-14,Partial Response
3,9249111.0,Anon35408,1962-01-18,1962.0,2015-09-12,53.648186,C15,Probably Related,Undifferentiated,Black or African American,...,N1,M1b,Computed Tomography,DECIPHER,Low risk,TID6018,Chemotherapy,2015-10-27,2023-08-21,Complete Response
4,2971959.0,Anon65131,1962-01-18,1962.0,2015-09-12,53.648186,C15,Probably Related,Undifferentiated,Black or African American,...,N1,M1b,Computed Tomography,DECIPHER,Low risk,TID6018,Chemotherapy,2015-10-27,2023-08-21,Complete Response
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,Anon43918,27/04/1941,1941.0,NaN,NaN,NaN,NaN,Undifferentiated,Other Pacific Islander,...,N1,Mx,Magnetic Resonance Imaging,DECIPHER,High risk,TID1596,Radiation Therapy,2015-12-13,2023-09-07,Complete Response
296,2533581.0,Anon63054,27/11/1992,1992.0,NaN,NaN,NaN,NaN,Not Disclosed,American Indian or Alaska Native,...,N1,Mx,Magnetic Resonance Imaging,ProMark,Intermediate risk,TID4389,Radiation Therapy,2015-09-12,2023-12-10,Complete Response
297,NaN,Anon34333,27/12/1983,1983.0,NaN,NaN,NaN,Definitely Related,Undifferentiated,Human,...,N1,M0,Magnetic Resonance Imaging,Oncotype DX Genomic Prostate Score,High risk,TID7273,Surgery,2017-11-28,2023-12-17,Partial Response
298,4924162.0,Anon85985,29/10/1998,1998.0,NaN,NaN,NaN,NaN,Female,Human,...,Nx,M1b,Computed Tomography,Prolaris,High risk,TID2744,XRT,2021-04-07,2024-07-05,Complete Response


In [2]:
print(dataset.columns)

Index(['Patient MRN', 'Anonymized Patient Identifier', 'Date of Birth',
       'Year of Birth', 'Date of Death', 'Age At Date of Death',
       'Cause of Death', 'Cause of Death Attributable to Treatment',
       'Sex at Birth', 'Patient Reported Race', 'Patient Reported Ethnicity',
       'Vital Status', 'Gender Identity', 'Has tobacco use history',
       'Height (cm)', 'Weight (kg)', 'Smoking Pack Years', 'Blood Pressure',
       'Heart Rate (bpm)', 'Temperature (Celsius)', 'On Clinical Trial',
       'Clinical Trial Numbers', 'Postal Code of Domicile', 'Education Level',
       'Primary Language Spoken at Home', 'Disability', 'Date of Record',
       'Age at Record', 'Staging System', 'ICD Version', 'ICD Code',
       'Histology', 'Staging-T Category', 'Staging-T', 'Staging-N',
       'Staging-M', 'Imaging for Biopsy', 'Pathology Molecular Test Name',
       'Pathology Molecular Test Result', 'Treatment Course ID',
       'Treatment Type', 'Start Date of Treatment', 'End Date of Tr

In [3]:
from datetime import datetime

In [4]:
# Step 1: Check for Missing 'Date of Birth' values
def validate_missing_dates(data):
    # Identify rows where 'Date of Birth' is missing
    missing_dob = data[data['Date of Birth'].isna()]
    
    # Report missing values
    if not missing_dob.empty:
        print("Rows with missing 'Date of Birth':")
        print(missing_dob[['Year of Birth', 'Date of Birth']])
    else:
        print("No missing 'Date of Birth' values found.")

# Step 2: Validate 'Date of Birth' format (ISO 8601)
def validate_dob_format_iso8601(data):
    invalid_format = []

    for index, row in data.iterrows():
        dob = row['Date of Birth']
        
        # Skip rows where 'Date of Birth' is already missing (NaT)
        if pd.isna(dob):
            continue
        
        # Try to parse the date using the ISO 8601 format
        try:
            # Check if the date is in the correct ISO format
            parsed_dob = datetime.strptime(dob, "%Y-%m-%dT%H:%M:%S%z")
        except Exception as e:
            # If there's an error in parsing, add the row to invalid format list
            invalid_format.append({
                'Row': index,
                'Year of Birth': row['Year of Birth'],
                'Date of Birth': dob,
                'Error': 'Invalid ISO 8601 format'
            })
    
    # Return the invalid rows as a DataFrame
    return pd.DataFrame(invalid_format)

# Step 3: Check for 'Date of Birth' values in the future
def validate_invalid_dates(data):
    # Convert the current date to a pandas Timestamp for comparison
    current_date = pd.Timestamp.now()

    # Convert 'Date of Birth' to datetime format (ignore errors to avoid crashing)
    data['Date of Birth'] = pd.to_datetime(data['Date of Birth'], errors='coerce')
    
    # Identify rows where 'Date of Birth' is greater than today's date (future date)
    future_dates = data[data['Date of Birth'] > current_date]
    
    # Report future dates
    if not future_dates.empty:
        print("Rows with 'Date of Birth' in the future:")
        print(future_dates[['Year of Birth', 'Date of Birth']])
    else:
        print("No future 'Date of Birth' values found.")

# Run the validation functions
validate_missing_dates(dataset)
invalid_dob_format = validate_dob_format_iso8601(dataset)
if not invalid_dob_format.empty:
    print("Rows with invalid 'Date of Birth' format:")
    print(invalid_dob_format)
validate_invalid_dates(dataset)

No missing 'Date of Birth' values found.
Rows with invalid 'Date of Birth' format:
     Row  Year of Birth Date of Birth                    Error
0      0            NaN    04/12/1996  Invalid ISO 8601 format
1      1         1983.0    14/08/1983  Invalid ISO 8601 format
2      2            NaN    15/09/1952  Invalid ISO 8601 format
3      3         1962.0    1962-01-18  Invalid ISO 8601 format
4      4         1962.0    1962-01-18  Invalid ISO 8601 format
..   ...            ...           ...                      ...
295  295         1941.0    27/04/1941  Invalid ISO 8601 format
296  296         1992.0    27/11/1992  Invalid ISO 8601 format
297  297         1983.0    27/12/1983  Invalid ISO 8601 format
298  298         1998.0    29/10/1998  Invalid ISO 8601 format
299  299            NaN    30/08/1931  Invalid ISO 8601 format

[300 rows x 4 columns]
No future 'Date of Birth' values found.


In [9]:
import pandas as pd

# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Ensure 'Date of Birth' is properly converted to datetime (with flexible format handling)
dataset['Date of Birth'] = pd.to_datetime(dataset['Date of Birth'], errors='coerce', dayfirst=True)

# Step 2: Check for missing 'Date of Birth' values again
def validate_missing_dates(data):
    # Identify rows where 'Date of Birth' is missing (after proper conversion)
    missing_dob = data[data['Date of Birth'].isna()]
    
    # Report missing values
    if not missing_dob.empty:
        print("Rows with missing 'Date of Birth':")
        print(missing_dob[['Anonymized Patient Identifier', 'Date of Birth']])
    else:
        print("No missing 'Date of Birth' values found.")

# Step 3: Run the updated validation function
validate_missing_dates(dataset)

Rows with missing 'Date of Birth':
    Anonymized Patient Identifier Date of Birth
3                       Anon35408           NaT
4                       Anon65131           NaT
5                       Anon35434           NaT
6                       Anon52420           NaT
7                       Anon35497           NaT
..                            ...           ...
282                     Anon89398           NaT
283                     Anon35438           NaT
284                     Anon83211           NaT
285                     Anon35482           NaT
286                     Anon86817           NaT

[251 rows x 2 columns]


In [10]:
# Temporarily allow pandas to display all rows
pd.set_option('display.max_rows', None)

# Display the entire 'Date of Birth' column
print(dataset[['Anonymized Patient Identifier', 'Date of Birth']])

# Reset to default row display limit after checking
pd.reset_option('display.max_rows')

    Anonymized Patient Identifier Date of Birth
0                       Anon70998    1996-12-04
1                             NaN    1983-08-14
2                       Anon98591    1952-09-15
3                       Anon35408           NaT
4                       Anon65131           NaT
5                       Anon35434           NaT
6                       Anon52420           NaT
7                       Anon35497           NaT
8                       Anon71046           NaT
9                       Anon35488           NaT
10                      Anon44354           NaT
11                      Anon35409           NaT
12                      Anon49274           NaT
13                            NaN    1943-03-28
14                      Anon35590    1995-02-03
15                      Anon58370    1975-07-14
16                      Anon35461           NaT
17                      Anon42568           NaT
18                      Anon35449           NaT
19                      Anon37568       

In [4]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Validate if the column name is correct
def validate_column_name(data, expected_column):
    actual_columns = data.columns
    if expected_column not in actual_columns:
        print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
        print(f"Found columns: {list(actual_columns)}")
    else:
        print(f"Column name '{expected_column}' is correct.")

# Step 2: Ensure 'Patient_DateOfBirth' is properly converted to datetime (with flexible format handling)
def validate_dob_format_iso8601(data):
    invalid_format = []
    
    for index, row in data.iterrows():
        dob = row['Patient_DateOfBirth']
        
        if pd.isna(dob):
            continue  # Skip missing values
        
        # Ensure it's in ISO 8601 format
        try:
            parsed_dob = datetime.strptime(dob.strftime("%Y-%m-%dT%H:%M:%S%z"), "%Y-%m-%dT%H:%M:%S%z")
        except Exception:
            invalid_format.append({
                'Row': index,
                'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                'Patient_DateOfBirth': dob,
                'Error': 'Invalid ISO 8601 format'
            })
    
    return pd.DataFrame(invalid_format)

# Step 3: Ensure time is set to midnight (00:00:00) for all valid dates
def set_midnight_time(data):
    data['Patient_DateOfBirth'] = data['Patient_DateOfBirth'].apply(lambda x: x.replace(hour=0, minute=0, second=0) if pd.notnull(x) else x)
    return data

# Step 4: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data):
    missing_dob = data[data['Patient_DateOfBirth'].isna()]
    if not missing_dob.empty:
        print("Rows with missing 'Patient_DateOfBirth':")
        print(missing_dob[['Anonymized Patient Identifier', 'Patient_DateOfBirth']])
    else:
        print("No missing 'Patient_DateOfBirth' values found.")

# Step 5: Check for duplicate 'Patient_DateOfBirth' entries per patient
def validate_duplicate_dob(data):
    duplicate_dob = data.duplicated(subset=['Anonymized Patient Identifier'], keep=False)
    if duplicate_dob.any():
        print("Duplicate DOBs found for some patients:")
        print(data[duplicate_dob][['Anonymized Patient Identifier', 'Patient_DateOfBirth']])
    else:
        print("No duplicate DOBs found.")

# Step 6: Validate future dates
def validate_invalid_dates(data):
    current_date = pd.Timestamp.now()
    future_dates = data[data['Patient_DateOfBirth'] > current_date]
    if not future_dates.empty:
        print("Rows with 'Patient_DateOfBirth' in the future:")
        print(future_dates[['Anonymized Patient Identifier', 'Patient_DateOfBirth']])
    else:
        print("No future 'Patient_DateOfBirth' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# First, validate the column name
validate_column_name(dataset, expected_column)

# If the column name is correct, continue with the rest of the validations
if expected_column in dataset.columns:
    dataset[expected_column] = pd.to_datetime(dataset[expected_column], errors='coerce', dayfirst=True)  # Ensure correct date format
    dataset = set_midnight_time(dataset)  # Ensure time is set to midnight
    
    validate_missing_dates(dataset)
    validate_duplicate_dob(dataset)
    
    invalid_dob_format = validate_dob_format_iso8601(dataset)
    if not invalid_dob_format.empty:
        print("Rows with invalid 'Patient_DateOfBirth' format:")
        print(invalid_dob_format)
    
    validate_invalid_dates(dataset)

Column name mismatch: Expected 'Patient_DateOfBirth', but not found in dataset.
Found columns: ['Patient MRN', 'Anonymized Patient Identifier', 'Date of Birth', 'Year of Birth', 'Date of Death', 'Age At Date of Death', 'Cause of Death', 'Cause of Death Attributable to Treatment', 'Sex at Birth', 'Patient Reported Race', 'Patient Reported Ethnicity', 'Vital Status', 'Gender Identity', 'Has tobacco use history', 'Height (cm)', 'Weight (kg)', 'Smoking Pack Years', 'Blood Pressure', 'Heart Rate (bpm)', 'Temperature (Celsius)', 'On Clinical Trial', 'Clinical Trial Numbers', 'Postal Code of Domicile', 'Education Level', 'Primary Language Spoken at Home', 'Disability', 'Date of Record', 'Age at Record', 'Staging System', 'ICD Version', 'ICD Code', 'Histology', 'Staging-T Category', 'Staging-T', 'Staging-N', 'Staging-M', 'Imaging for Biopsy', 'Pathology Molecular Test Name', 'Pathology Molecular Test Result', 'Treatment Course ID', 'Treatment Type', 'Start Date of Treatment', 'End Date of Trea

In [5]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth'
def find_equivalent_column(data, expected_column):
    # Look for similar column names
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}'. Proceeding with validation.")
            return col  # Return the recognized column
    # If no matching column is found, flag the mismatch
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Ensure 'Patient_DateOfBirth' is properly converted to datetime (with flexible format handling)
def validate_dob_format_iso8601(data, dob_column):
    invalid_format = []
    
    for index, row in data.iterrows():
        dob = row[dob_column]
        
        if pd.isna(dob):
            continue  # Skip missing values
        
        # Ensure it's in ISO 8601 format
        try:
            parsed_dob = datetime.strptime(dob.strftime("%Y-%m-%dT%H:%M:%S%z"), "%Y-%m-%dT%H:%M:%S%z")
        except Exception:
            invalid_format.append({
                'Row': index,
                'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                dob_column: dob,
                'Error': 'Invalid ISO 8601 format'
            })
    
    return pd.DataFrame(invalid_format)

# Step 3: Ensure time is set to midnight (00:00:00) for all valid dates
def set_midnight_time(data, dob_column):
    data[dob_column] = data[dob_column].apply(lambda x: x.replace(hour=0, minute=0, second=0) if pd.notnull(x) else x)
    return data

# Step 4: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Step 5: Check for duplicate 'Patient_DateOfBirth' entries per patient
def validate_duplicate_dob(data, dob_column):
    duplicate_dob = data.duplicated(subset=['Anonymized Patient Identifier'], keep=False)
    if duplicate_dob.any():
        print(f"Duplicate DOBs found for some patients in '{dob_column}':")
        print(data[duplicate_dob][['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No duplicate DOBs found in '{dob_column}'.")

# Step 6: Validate future dates
def validate_invalid_dates(data, dob_column):
    current_date = pd.Timestamp.now()
    future_dates = data[data[dob_column] > current_date]
    if not future_dates.empty:
        print(f"Rows with '{dob_column}' in the future:")
        print(future_dates[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No future '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    dataset[recognized_column] = pd.to_datetime(dataset[recognized_column], errors='coerce', dayfirst=True)  # Ensure correct date format
    dataset = set_midnight_time(dataset, recognized_column)  # Ensure time is set to midnight
    
    validate_missing_dates(dataset, recognized_column)
    validate_duplicate_dob(dataset, recognized_column)
    
    invalid_dob_format = validate_dob_format_iso8601(dataset, recognized_column)
    if not invalid_dob_format.empty:
        print(f"Rows with invalid '{recognized_column}' format:")
        print(invalid_dob_format)
    
    validate_invalid_dates(dataset, recognized_column)

Recognized 'Date of Birth' as 'Patient_DateOfBirth'. Proceeding with validation.
Rows with missing 'Date of Birth':
    Anonymized Patient Identifier Date of Birth
3                       Anon35408           NaT
4                       Anon65131           NaT
5                       Anon35434           NaT
6                       Anon52420           NaT
7                       Anon35497           NaT
..                            ...           ...
282                     Anon89398           NaT
283                     Anon35438           NaT
284                     Anon83211           NaT
285                     Anon35482           NaT
286                     Anon86817           NaT

[251 rows x 2 columns]
Duplicate DOBs found for some patients in 'Date of Birth':
    Anonymized Patient Identifier Date of Birth
1                             NaN    1983-08-14
13                            NaN    1943-03-28
67                            NaN           NaT
69                            NaN

In [6]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    # Look for similar column names
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    # If no matching column is found, flag the mismatch
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Ensure 'Patient_DateOfBirth' is properly converted to datetime (with flexible format handling)
def validate_dob_format_iso8601(data, dob_column):
    invalid_format = []
    
    for index, row in data.iterrows():
        dob = row[dob_column]
        
        if pd.isna(dob):
            continue  # Skip missing values
        
        # Ensure it's in ISO 8601 format
        try:
            parsed_dob = datetime.strptime(dob.strftime("%Y-%m-%dT%H:%M:%S%z"), "%Y-%m-%dT%H:%M:%S%z")
        except Exception:
            invalid_format.append({
                'Row': index,
                'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                dob_column: dob,
                'Error': 'Invalid ISO 8601 format'
            })
    
    return pd.DataFrame(invalid_format)

# Step 3: Ensure time is set to midnight (00:00:00) for all valid dates
def set_midnight_time(data, dob_column):
    data[dob_column] = data[dob_column].apply(lambda x: x.replace(hour=0, minute=0, second=0) if pd.notnull(x) else x)
    return data

# Step 4: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Step 5: Check for duplicate 'Patient_DateOfBirth' entries per patient
def validate_duplicate_dob(data, dob_column):
    duplicate_dob = data.duplicated(subset=['Anonymized Patient Identifier'], keep=False)
    if duplicate_dob.any():
        print(f"Duplicate DOBs found for some patients in '{dob_column}':")
        print(data[duplicate_dob][['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No duplicate DOBs found in '{dob_column}'.")

# Step 6: Validate future dates
def validate_invalid_dates(data, dob_column):
    current_date = pd.Timestamp.now()
    future_dates = data[data[dob_column] > current_date]
    if not future_dates.empty:
        print(f"Rows with '{dob_column}' in the future:")
        print(future_dates[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No future '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    dataset[recognized_column] = pd.to_datetime(dataset[recognized_column], errors='coerce', dayfirst=True)  # Ensure correct date format
    dataset = set_midnight_time(dataset, recognized_column)  # Ensure time is set to midnight
    
    validate_missing_dates(dataset, recognized_column)
    validate_duplicate_dob(dataset, recognized_column)
    
    invalid_dob_format = validate_dob_format_iso8601(dataset, recognized_column)
    if not invalid_dob_format.empty:
        print(f"Rows with invalid '{recognized_column}' format:")
        print(invalid_dob_format)
    
    validate_invalid_dates(dataset, recognized_column)

    # Output a final message indicating the column name was incorrect
    if recognized_column != expected_column:
        print(f"Note: The column '{recognized_column}' was used, but it does not match the expected name '{expected_column}'.")


# Problem with this output is that the "Date of Birth" values in the dataset are stored as strings, rather than being properly recognized as datetime objects.

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.
Rows with missing 'Date of Birth':
    Anonymized Patient Identifier Date of Birth
3                       Anon35408           NaT
4                       Anon65131           NaT
5                       Anon35434           NaT
6                       Anon52420           NaT
7                       Anon35497           NaT
..                            ...           ...
282                     Anon89398           NaT
283                     Anon35438           NaT
284                     Anon83211           NaT
285                     Anon35482           NaT
286                     Anon86817           NaT

[251 rows x 2 columns]
Duplicate DOBs found for some patients in 'Date of Birth':
    Anonymized Patient Identifier Date of Birth
1                             NaN    1983-08-14
13                            NaN    1943-03-28
67                            NaN           NaT
69          

In [7]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Ensure 'Patient_DateOfBirth' is properly converted to datetime
def convert_to_datetime(data, dob_column):
    data[dob_column] = pd.to_datetime(data[dob_column], errors='coerce', dayfirst=True)  # Force day-first format
    return data

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert 'Date of Birth' to datetime, forcing day-first format
    dataset = convert_to_datetime(dataset, recognized_column)

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.
Rows with missing 'Date of Birth':
    Anonymized Patient Identifier Date of Birth
3                       Anon35408           NaT
4                       Anon65131           NaT
5                       Anon35434           NaT
6                       Anon52420           NaT
7                       Anon35497           NaT
..                            ...           ...
282                     Anon89398           NaT
283                     Anon35438           NaT
284                     Anon83211           NaT
285                     Anon35482           NaT
286                     Anon86817           NaT

[251 rows x 2 columns]


In [8]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Check if 'Date of Birth' is in string format and flag it
def flag_string_format_dates(data, dob_column):
    invalid_dob = []

    # Loop through the dataset and check if the value is not datetime
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string or invalid
        if isinstance(dob, str):
            invalid_dob.append({
                'Row': index,
                'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                dob_column: dob,
                'Error': 'Date is in string format'
            })
    
    # Return the invalid rows as a DataFrame
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Flag rows where 'Date of Birth' is still in string format
    invalid_dates = flag_string_format_dates(dataset, recognized_column)
    if not invalid_dates.empty:
        print(f"Rows with 'Date of Birth' in string format:")
        print(invalid_dates)

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.
Rows with 'Date of Birth' in string format:
     Row Anonymized Patient Identifier Date of Birth                     Error
0      0                     Anon70998    04/12/1996  Date is in string format
1      1                           NaN    14/08/1983  Date is in string format
2      2                     Anon98591    15/09/1952  Date is in string format
3      3                     Anon35408    1962-01-18  Date is in string format
4      4                     Anon65131    1962-01-18  Date is in string format
..   ...                           ...           ...                       ...
295  295                     Anon43918    27/04/1941  Date is in string format
296  296                     Anon63054    27/11/1992  Date is in string format
297  297                     Anon34333    27/12/1983  Date is in string format
298  298                     Anon85985    29/10/1998  Date is in

In [9]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Convert correctly formatted ISO 8601 strings to datetime, flag invalid strings
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []

    # Try to convert each value to datetime if it's a string
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string and try to convert to datetime
        if isinstance(dob, str):
            try:
                # Attempt to parse the date (ensure it's in ISO 8601)
                parsed_dob = pd.to_datetime(dob, format="%Y-%m-%dT%H:%M:%SZ", errors='raise')
                # If conversion is successful, replace the string with the datetime object
                data.at[index, dob_column] = parsed_dob
            except Exception:
                # If parsing fails, flag the error
                invalid_dob.append({
                    'Row': index,
                    'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                    dob_column: dob,
                    'Error': 'Invalid ISO 8601 format'
                })

    # Return any invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert correctly formatted strings to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    if not invalid_dates.empty:
        print(f"Rows with 'Date of Birth' that have invalid ISO 8601 format:")
        print(invalid_dates)

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

# Looking good but all rows are getting an error because the time isn't included. Next iteration will bypass this.

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.
Rows with 'Date of Birth' that have invalid ISO 8601 format:
     Row Anonymized Patient Identifier Date of Birth                    Error
0      0                     Anon70998    04/12/1996  Invalid ISO 8601 format
1      1                           NaN    14/08/1983  Invalid ISO 8601 format
2      2                     Anon98591    15/09/1952  Invalid ISO 8601 format
3      3                     Anon35408    1962-01-18  Invalid ISO 8601 format
4      4                     Anon65131    1962-01-18  Invalid ISO 8601 format
..   ...                           ...           ...                      ...
295  295                     Anon43918    27/04/1941  Invalid ISO 8601 format
296  296                     Anon63054    27/11/1992  Invalid ISO 8601 format
297  297                     Anon34333    27/12/1983  Invalid ISO 8601 format
298  298                     Anon85985    29/10/1998  Inv

In [10]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Convert correctly formatted ISO 8601 strings (including date-only format) to datetime, flag invalid strings
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []

    # Try to convert each value to datetime if it's a string
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string and try to convert to datetime
        if isinstance(dob, str):
            try:
                # Attempt to parse the date (allowing both date-only and datetime format)
                parsed_dob = pd.to_datetime(dob, format="%Y-%m-%d", errors='raise')  # Allow date-only format
                # If conversion is successful, replace the string with the datetime object
                data.at[index, dob_column] = parsed_dob
            except Exception:
                # If parsing fails, flag the error
                invalid_dob.append({
                    'Row': index,
                    'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                    dob_column: dob,
                    'Error': 'Invalid ISO 8601 format'
                })

    # Return any invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert correctly formatted strings (including date-only) to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    if not invalid_dates.empty:
        print(f"Rows with 'Date of Birth' that have invalid ISO 8601 format:")
        print(invalid_dates)

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.
Rows with 'Date of Birth' that have invalid ISO 8601 format:
    Row Anonymized Patient Identifier Date of Birth                    Error
0     0                     Anon70998    04/12/1996  Invalid ISO 8601 format
1     1                           NaN    14/08/1983  Invalid ISO 8601 format
2     2                     Anon98591    15/09/1952  Invalid ISO 8601 format
3    13                           NaN    28/03/1943  Invalid ISO 8601 format
4    14                     Anon35590    03/02/1995  Invalid ISO 8601 format
5    15                     Anon58370    14/07/1975  Invalid ISO 8601 format
6    43                     Anon74403    26/01/1941  Invalid ISO 8601 format
7    44                     Anon44632    05/10/1996  Invalid ISO 8601 format
8    45                     Anon32379    11/11/1978  Invalid ISO 8601 format
9    46                     Anon50990    14/05/1971  Invalid ISO 86

In [11]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Convert correctly formatted ISO 8601 strings (including date-only format) to datetime, flag invalid strings
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []

    # Try to convert each value to datetime if it's a string
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string and try to convert to datetime
        if isinstance(dob, str):
            try:
                # Attempt to parse the date (allowing both date-only and datetime format)
                parsed_dob = pd.to_datetime(dob, format="%Y-%m-%d", errors='raise')  # Allow date-only format
                # If conversion is successful, replace the string with the datetime object
                data.at[index, dob_column] = parsed_dob
            except Exception:
                # If parsing fails, flag the error
                invalid_dob.append({
                    'Row': index,
                    'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                    dob_column: dob,
                    'Error': 'Invalid ISO 8601 format'
                })

    # Return the DataFrame with invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert correctly formatted strings (including date-only) to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    
    # Output the number of invalid rows
    invalid_count = len(invalid_dates)
    if invalid_count > 0:
        print(f"\n{invalid_count} rows have an invalid ISO 8601 format:\n")
        print(invalid_dates)
    else:
        print("\nNo rows with invalid ISO 8601 format found.\n")

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

#shows the total number of incorrectly formatted rows in the output.

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.

49 rows have an invalid ISO 8601 format:

    Row Anonymized Patient Identifier Date of Birth                    Error
0     0                     Anon70998    04/12/1996  Invalid ISO 8601 format
1     1                           NaN    14/08/1983  Invalid ISO 8601 format
2     2                     Anon98591    15/09/1952  Invalid ISO 8601 format
3    13                           NaN    28/03/1943  Invalid ISO 8601 format
4    14                     Anon35590    03/02/1995  Invalid ISO 8601 format
5    15                     Anon58370    14/07/1975  Invalid ISO 8601 format
6    43                     Anon74403    26/01/1941  Invalid ISO 8601 format
7    44                     Anon44632    05/10/1996  Invalid ISO 8601 format
8    45                     Anon32379    11/11/1978  Invalid ISO 8601 format
9    46                     Anon50990    14/05/1971  Invalid ISO 8601 format
10   68 

In [20]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Convert correctly formatted ISO 8601 strings (including date-only format) to datetime, flag invalid strings
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []

    # Try to convert each value to datetime if it's a string
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string and try to convert to datetime
        if isinstance(dob, str):
            try:
                # Attempt to parse the date (allowing both date-only and datetime format)
                parsed_dob = pd.to_datetime(dob, format="%Y-%m-%d", errors='raise')  # Allow date-only format
                # If conversion is successful, replace the string with the datetime object
                data.at[index, dob_column] = parsed_dob
            except Exception:
                # If parsing fails, flag the error
                invalid_dob.append({
                    'Row': index,
                    'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                    dob_column: dob,
                    'Error': 'Invalid ISO 8601 format'
                })

    # Return the DataFrame with invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert correctly formatted strings (including date-only) to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    
    # Output the number of invalid rows
    invalid_count = len(invalid_dates)
    if invalid_count > 0:
        # Extract just the row numbers
        invalid_row_numbers = invalid_dates['Row'].tolist()
        invalid_row_numbers_str = ','.join(map(str, invalid_row_numbers))

        # Display the count and the row numbers
        print(f"\n{invalid_count} rows have an invalid ISO 8601 format:")
        print(f"Row numbers: {invalid_row_numbers_str}\n")

        # Output the detailed list of invalid rows
        print(invalid_dates.to_string(index=False))  # Output invalid rows without index
    else:
        print("\nNo rows with invalid ISO 8601 format found.\n")

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

#now lists all the rows with invalid ISO 8601 format

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.

49 rows have an invalid ISO 8601 format:
Row numbers: 0,1,2,13,14,15,43,44,45,46,68,69,70,95,96,97,98,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,274,287,288,289,290,291,292,293,294,295,296,297,298,299

 Row Anonymized Patient Identifier Date of Birth                   Error
   0                     Anon70998    04/12/1996 Invalid ISO 8601 format
   1                           NaN    14/08/1983 Invalid ISO 8601 format
   2                     Anon98591    15/09/1952 Invalid ISO 8601 format
  13                           NaN    28/03/1943 Invalid ISO 8601 format
  14                     Anon35590    03/02/1995 Invalid ISO 8601 format
  15                     Anon58370    14/07/1975 Invalid ISO 8601 format
  43                     Anon74403    26/01/1941 Invalid ISO 8601 format
  44                     Anon44632    05/10/1996 Invalid ISO 8601 format
  45     

In [15]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Step 2: Convert correctly formatted ISO 8601 strings (including date-only format) to datetime, flag invalid strings
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []

    # Try to convert each value to datetime if it's a string
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string and try to convert to datetime
        if isinstance(dob, str):
            try:
                # Attempt to parse the date (allowing both date-only and datetime format)
                parsed_dob = pd.to_datetime(dob, format="%Y-%m-%d", errors='raise')  # Allow date-only format
                # If conversion is successful, replace the string with the datetime object
                data.at[index, dob_column] = parsed_dob
            except Exception:
                # If parsing fails, flag the error
                invalid_dob.append({
                    'Row': index,
                    'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                    dob_column: dob,
                    'Error': 'Invalid ISO 8601 format'
                })

    # Return the DataFrame with invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Step 4: Check for duplicate 'Date of Birth' entries
def check_duplicates(data, dob_column):
    duplicates = data[data.duplicated(subset=[dob_column], keep=False)]
    
    if not duplicates.empty:
        print(f"\nDuplicate 'Date of Birth' entries found:")
        print(duplicates[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"\nNo duplicate 'Date of Birth' values found.\n")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert correctly formatted strings (including date-only) to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    
    # Output the number of invalid rows
    invalid_count = len(invalid_dates)
    if invalid_count > 0:
        # Extract just the row numbers
        invalid_row_numbers = invalid_dates['Row'].tolist()
        invalid_row_numbers_str = ','.join(map(str, invalid_row_numbers))

        # Display the count and the row numbers
        print(f"\n{invalid_count} rows have an invalid ISO 8601 format:")
        print(f"Row numbers: {invalid_row_numbers_str}\n")

        # Output the detailed list of invalid rows
        print(invalid_dates.to_string(index=False))  # Output invalid rows without index
    else:
        print("\nNo rows with invalid ISO 8601 format found.\n")

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

    # Check for duplicate 'Date of Birth' entries
    check_duplicates(dataset, recognized_column)

#added logic to check for duplicate entries. incorrectly flagging duplicate entries due to the way date values are represented.

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.

49 rows have an invalid ISO 8601 format:
Row numbers: 0,1,2,13,14,15,43,44,45,46,68,69,70,95,96,97,98,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,274,287,288,289,290,291,292,293,294,295,296,297,298,299

 Row Anonymized Patient Identifier Date of Birth                   Error
   0                     Anon70998    04/12/1996 Invalid ISO 8601 format
   1                           NaN    14/08/1983 Invalid ISO 8601 format
   2                     Anon98591    15/09/1952 Invalid ISO 8601 format
  13                           NaN    28/03/1943 Invalid ISO 8601 format
  14                     Anon35590    03/02/1995 Invalid ISO 8601 format
  15                     Anon58370    14/07/1975 Invalid ISO 8601 format
  43                     Anon74403    26/01/1941 Invalid ISO 8601 format
  44                     Anon44632    05/10/1996 Invalid ISO 8601 format
  45     

In [19]:
# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"Found columns: {list(data.columns)}")
    return None

# Debugging: Print the first few values of the Date of Birth column to inspect the format
def inspect_raw_dates(data, dob_column):
    print("\nInspecting the first few raw values in the 'Date of Birth' column:")
    print(data[dob_column].head(10))  # Print the first 10 entries to check the format

# Step 2: Convert dates with flexible parsing for different formats
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []
    
    # Print raw dates for debugging
    inspect_raw_dates(data, dob_column)

    # Try to convert multiple possible formats, falling back to 'coerce'
    data[dob_column] = pd.to_datetime(data[dob_column], errors='coerce')

    for index, row in data.iterrows():
        dob = row[dob_column]

        if pd.isna(dob):
            # If conversion fails and results in NaT, flag the invalid entry
            invalid_dob.append({
                'Row': index,
                'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                dob_column: row[dob_column],
                'Error': 'Invalid ISO 8601 format'
            })

    # Return the DataFrame with invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"No missing '{dob_column}' values found.")

# Step 4: Check for duplicate 'Date of Birth' entries (based only on the date part)
def check_duplicates(data, dob_column):
    # Ensure the column is datetime, then extract the date part only for comparison
    data[dob_column] = pd.to_datetime(data[dob_column], errors='coerce')  # Ensure datetime conversion
    data['Date_Only'] = data[dob_column].dt.date  # Extract just the date part
    
    duplicates = data[data.duplicated(subset=['Date_Only'], keep=False)]

    if not duplicates.empty:
        print(f"\nDuplicate 'Date of Birth' entries found:")
        print(duplicates[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"\nNo duplicate 'Date of Birth' values found.\n")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert correctly formatted strings (including date-only) to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    
    # Output the number of invalid rows
    invalid_count = len(invalid_dates)
    if invalid_count > 0:
        # Extract just the row numbers
        invalid_row_numbers = invalid_dates['Row'].tolist()
        invalid_row_numbers_str = ','.join(map(str, invalid_row_numbers))

        # Display the count and the row numbers
        print(f"\n{invalid_count} rows have an invalid ISO 8601 format:")
        print(f"Row numbers: {invalid_row_numbers_str}\n")

        # Output the detailed list of invalid rows
        print(invalid_dates.to_string(index=False))  # Output invalid rows without index
    else:
        print("\nNo rows with invalid ISO 8601 format found.\n")

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

    # Check for duplicate 'Date of Birth' entries
    check_duplicates(dataset, recognized_column)

#completely messes it up when i try to validate for duplication.

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.

Inspecting the first few raw values in the 'Date of Birth' column:
0    04/12/1996
1    14/08/1983
2    15/09/1952
3    1962-01-18
4    1962-01-18
5    1977-09-03
6    1977-09-03
7    1979-10-29
8    1979-10-29
9    1981-03-30
Name: Date of Birth, dtype: object

284 rows have an invalid ISO 8601 format:
Row numbers: 1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,1

In [24]:
import pandas as pd
from colorama import Fore, Style, init
from datetime import datetime

# Initialize colorama for coloring in the terminal
init(autoreset=True)

# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"{Fore.RED}Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"{Fore.RED}Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"{Fore.RED}Found columns: {list(data.columns)}")
    return None

# Step 2: Convert correctly formatted ISO 8601 strings (including date-only format) to datetime, flag invalid strings
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []

    # Try to convert each value to datetime if it's a string
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string and try to convert to datetime
        if isinstance(dob, str):
            try:
                # Attempt to parse the date (allowing both date-only and datetime format)
                parsed_dob = pd.to_datetime(dob, format="%Y-%m-%d", errors='raise')  # Allow date-only format
                # If conversion is successful, replace the string with the datetime object
                data.at[index, dob_column] = parsed_dob
            except Exception:
                # If parsing fails, flag the error
                invalid_dob.append({
                    'Row': index,
                    'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                    dob_column: dob,
                    'Error': 'Invalid ISO 8601 format'
                })

    # Return the DataFrame with invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"{Fore.RED}Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"{Fore.GREEN}No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Convert correctly formatted strings (including date-only) to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    
    # Output the number of invalid rows
    invalid_count = len(invalid_dates)
    if invalid_count > 0:
        # Extract just the row numbers
        invalid_row_numbers = invalid_dates['Row'].tolist()
        invalid_row_numbers_str = ','.join(map(str, invalid_row_numbers))

        # Display the count and the row numbers
        print(f"{Fore.RED}\n{invalid_count} rows have an invalid ISO 8601 format:")
        print(f"Row numbers: {invalid_row_numbers_str}\n")

        # Output the detailed list of invalid rows
        print(invalid_dates.to_string(index=False))  # Output invalid rows without index
    else:
        print(f"{Fore.GREEN}\nNo rows with invalid ISO 8601 format found.\n")

    # Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.

49 rows have an invalid ISO 8601 format:
Row numbers: 0,1,2,13,14,15,43,44,45,46,68,69,70,95,96,97,98,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,274,287,288,289,290,291,292,293,294,295,296,297,298,299

 Row Anonymized Patient Identifier Date of Birth                   Error
   0                     Anon70998    04/12/1996 Invalid ISO 8601 format
   1                           NaN    14/08/1983 Invalid ISO 8601 format
   2                     Anon98591    15/09/1952 Invalid ISO 8601 format
  13                           NaN    28/03/1943 Invalid ISO 8601 format
  14                     Anon35590    03/02/1995 Invalid ISO 8601 format
  15                     Anon58370    14/07/1975 Invalid ISO 8601 format
  43                     Anon74403    26/01/1941 Invalid ISO 8601 format
  44                     Anon44632    05/10/1996 Invalid ISO 8601 format
  45     

In [25]:
import pandas as pd
from colorama import Fore, init

# Initialize colorama (if needed for your environment)
init(autoreset=True)

# Load the dataset
dataset = pd.read_excel('Hospital Patient Dataset Validata.xlsx')

# Step 1: Function to recognize 'Date of Birth' as 'Patient_DateOfBirth' and flag incorrect names
def find_equivalent_column(data, expected_column):
    alternative_names = ["Date of Birth", "DOB", "Birthdate"]  # Add more alternatives as needed
    for col in data.columns:
        if col in alternative_names:
            print(f"1. {Fore.RED}Recognized '{col}' as '{expected_column}' for validation, but the column name is incorrect.")
            return col  # Return the recognized column
    print(f"1. {Fore.RED}Column name mismatch: Expected '{expected_column}', but not found in dataset.")
    print(f"   Found columns: {list(data.columns)}")
    return None

# Step 2: Convert correctly formatted ISO 8601 strings (including date-only format) to datetime, flag invalid strings
def convert_or_flag_dates(data, dob_column):
    invalid_dob = []

    # Try to convert each value to datetime if it's a string
    for index, row in data.iterrows():
        dob = row[dob_column]

        # Check if the type of 'dob' is string and try to convert to datetime
        if isinstance(dob, str):
            try:
                # Attempt to parse the date (allowing both date-only and datetime format)
                parsed_dob = pd.to_datetime(dob, format="%Y-%m-%d", errors='raise')  # Allow date-only format
                # If conversion is successful, replace the string with the datetime object
                data.at[index, dob_column] = parsed_dob
            except Exception:
                # If parsing fails, flag the error
                invalid_dob.append({
                    'Row': index,
                    'Anonymized Patient Identifier': row['Anonymized Patient Identifier'],
                    dob_column: dob,
                    'Error': 'Invalid ISO 8601 format'
                })

    # Return the DataFrame with invalid rows
    return pd.DataFrame(invalid_dob)

# Step 3: Check for missing 'Patient_DateOfBirth' values
def validate_missing_dates(data, dob_column):
    missing_dob = data[data[dob_column].isna()]
    if not missing_dob.empty:
        print(f"3. {Fore.RED}Rows with missing '{dob_column}':")
        print(missing_dob[['Anonymized Patient Identifier', dob_column]])
    else:
        print(f"3. {Fore.GREEN}No missing '{dob_column}' values found.")

# Run the validation functions
expected_column = 'Patient_DateOfBirth'

# Step 1: Recognize and handle equivalent columns for 'Patient_DateOfBirth'
recognized_column = find_equivalent_column(dataset, expected_column)

# If an equivalent column is recognized, proceed with the validation
if recognized_column:
    # Step 2: Convert correctly formatted strings (including date-only) to datetime, flag invalid strings
    invalid_dates = convert_or_flag_dates(dataset, recognized_column)
    
    # Output the number of invalid rows
    invalid_count = len(invalid_dates)
    if invalid_count > 0:
        # Extract just the row numbers
        invalid_row_numbers = invalid_dates['Row'].tolist()
        invalid_row_numbers_str = ','.join(map(str, invalid_row_numbers))

        # Display the count and the row numbers
        print(f"2. {Fore.RED}{invalid_count} rows have an invalid ISO 8601 format:")
        print(f"   Row numbers: {invalid_row_numbers_str}\n")

        # Output the detailed list of invalid rows
        print(invalid_dates.to_string(index=False))  # Output invalid rows without index
    else:
        print(f"2. {Fore.GREEN}No rows with invalid ISO 8601 format found.\n")

    # Step 3: Re-run the missing value check
    validate_missing_dates(dataset, recognized_column)

1. Recognized 'Date of Birth' as 'Patient_DateOfBirth' for validation, but the column name is incorrect.
2. 49 rows have an invalid ISO 8601 format:
   Row numbers: 0,1,2,13,14,15,43,44,45,46,68,69,70,95,96,97,98,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,274,287,288,289,290,291,292,293,294,295,296,297,298,299

 Row Anonymized Patient Identifier Date of Birth                   Error
   0                     Anon70998    04/12/1996 Invalid ISO 8601 format
   1                           NaN    14/08/1983 Invalid ISO 8601 format
   2                     Anon98591    15/09/1952 Invalid ISO 8601 format
  13                           NaN    28/03/1943 Invalid ISO 8601 format
  14                     Anon35590    03/02/1995 Invalid ISO 8601 format
  15                     Anon58370    14/07/1975 Invalid ISO 8601 format
  43                     Anon74403    26/01/1941 Invalid ISO 8601 format
  44                     Anon44632    05/10/1996 Invalid ISO 8601 format
 